# First Draft of IVS Score Pipeline (3007)

In [24]:
import pandas as pd

# Load data from the excel into a DataFrame
file_path = 'full_length_two_per_bin.xlsx'
df = pd.read_excel(file_path)

# display(df)

In [26]:
from scipy.stats import ks_2samp

# Ensure all data is numeric
df = df.apply(pd.to_numeric, errors='coerce')

# Function to compute pairwise Kolmogorov-Smirnov test
def ks_test(dataframe):
    vendors = dataframe.columns[1:]
    ks_results = pd.DataFrame(index=vendors, columns=vendors)
    for i in vendors:
        for j in vendors:
            ks_stat, ks_pvalue = ks_2samp(dataframe[i], dataframe[j])
            ks_results.loc[i, j] = ks_stat
    return ks_results

# Compute the Kolmogorov-Smirnov test results
ks_results = ks_test(df)

# # Display the results
# print("Kolmogorov-Smirnov Test Results:")
# print(ks_results)

In [30]:
# Filter the columns to include only the specified titles
titles_to_keep = ["ESET_NOD32", "Microsoft", "Kaspersky", "Avira", "BitDefender", "TrendMicro"]

ks_results_filtered = ks_results[titles_to_keep]

# Display the filtered results
print("Filtered Kolmogorov-Smirnov Test Results:")
print(ks_results_filtered)

Filtered Kolmogorov-Smirnov Test Results:
             ESET_NOD32 Microsoft Kaspersky     Avira BitDefender TrendMicro
ALYac          0.243243  0.189189  0.216216  0.135135    0.081081   0.432432
APEX            0.27027  0.162162  0.162162  0.108108    0.243243   0.432432
AVG            0.162162  0.189189   0.27027  0.189189    0.135135   0.486486
Acronis        0.891892  0.837838   0.72973  0.783784    0.675676   0.459459
AhnLab_V3      0.324324  0.162162  0.108108  0.135135    0.162162   0.378378
...                 ...       ...       ...       ...         ...        ...
Zillya         0.540541  0.378378  0.297297  0.405405    0.378378   0.405405
ZoneAlarm      0.297297  0.135135  0.162162  0.135135    0.162162   0.378378
Zoner          0.945946  0.891892  0.810811  0.864865    0.756757   0.567568
alibabacloud   0.135135  0.216216  0.324324  0.216216    0.189189   0.540541
tehtris        0.891892  0.783784  0.648649   0.72973    0.648649   0.405405

[78 rows x 6 columns]


In [29]:
# Calculate the pre_1, pre_2, and Score columns
ks_results_filtered['pre_1'] = (6 * ks_results_filtered['ESET_NOD32'] +
                                5 * ks_results_filtered['Microsoft'] +
                                4 * ks_results_filtered['Kaspersky'] +
                                3 * ks_results_filtered['Avira'] +
                                2 * ks_results_filtered['BitDefender'] +
                                1 * ks_results_filtered['TrendMicro'])

ks_results_filtered['pre_2'] = 1 / ks_results_filtered['pre_1']
ks_results_filtered['Score'] = 10 * ks_results_filtered['pre_2']

# Calculate the sum of all values in the Score column
score_sum = ks_results_filtered['Score'].sum()

# Print the resultant dataframe and the sum of scores
print(ks_results_filtered)
print("Sum of all values in the Score column:", score_sum)

             ESET_NOD32 Microsoft Kaspersky     Avira BitDefender TrendMicro  \
ALYac          0.243243  0.189189  0.216216  0.135135    0.081081   0.432432   
APEX            0.27027  0.162162  0.162162  0.108108    0.243243   0.432432   
AVG            0.162162  0.189189   0.27027  0.189189    0.135135   0.486486   
Acronis        0.891892  0.837838   0.72973  0.783784    0.675676   0.459459   
AhnLab_V3      0.324324  0.162162  0.108108  0.135135    0.162162   0.378378   
...                 ...       ...       ...       ...         ...        ...   
Zillya         0.540541  0.378378  0.297297  0.405405    0.378378   0.405405   
ZoneAlarm      0.297297  0.135135  0.162162  0.135135    0.162162   0.378378   
Zoner          0.945946  0.891892  0.810811  0.864865    0.756757   0.567568   
alibabacloud   0.135135  0.216216  0.324324  0.216216    0.189189   0.540541   
tehtris        0.891892  0.783784  0.648649   0.72973    0.648649   0.405405   

                  pre_1     pre_2     S

C:\Users\t-jiajunfu\AppData\Local\Temp\ipykernel_13032\2891214462.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ks_results_filtered['pre_1'] = (6 * ks_results_filtered['ESET_NOD32'] +
C:\Users\t-jiajunfu\AppData\Local\Temp\ipykernel_13032\2891214462.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ks_results_filtered['pre_2'] = 1 / ks_results_filtered['pre_1']
C:\Users\t-jiajunfu\AppData\Local\Temp\ipykernel_13032\2891214462.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [14]:
# Save the classification report to an Excel file
ks_results_filtered.to_excel("0720_score.xlsx", index=True)

print("Classification report saved to 0720_score.xlsx")

Classification report saved to 0720_score.xlsx


    Generate the format of part of the Kusto query

In [32]:
# Rename the dataframe
df = ks_results_filtered

# Generate the scoring logic
scoring_logic = ""
for vendor in df.index:
    vendor_formatted = vendor.replace('_', '-')
    # score = vendor_scores.get(vendor_formatted, 0)
    score = round(df.loc[vendor, 'Score'], 2)
    scoring_logic += f"    {score} * iif(Detections has '{vendor_formatted}:', 1, 0) +\n"

# Remove the last '+' and '\n'
scoring_logic = scoring_logic.rstrip(' +\n') + "\n"

# Display the scoring logic
print(scoring_logic)


    2.34 * iif(Detections has 'ALYac:', 1, 0) +
    2.31 * iif(Detections has 'APEX:', 1, 0) +
    2.31 * iif(Detections has 'AVG:', 1, 0) +
    0.6 * iif(Detections has 'Acronis:', 1, 0) +
    2.33 * iif(Detections has 'AhnLab-V3:', 1, 0) +
    1.42 * iif(Detections has 'Alibaba:', 1, 0) +
    1.77 * iif(Detections has 'Antiy-AVL:', 1, 0) +
    2.34 * iif(Detections has 'Arcabit:', 1, 0) +
    2.31 * iif(Detections has 'Avast:', 1, 0) +
    0.61 * iif(Detections has 'Avast-Mobile:', 1, 0) +
    2.66 * iif(Detections has 'Avira:', 1, 0) +
    0.62 * iif(Detections has 'Baidu:', 1, 0) +
    2.47 * iif(Detections has 'BitDefender:', 1, 0) +
    0.64 * iif(Detections has 'BitDefenderFalx:', 1, 0) +
    0.78 * iif(Detections has 'BitDefenderTheta:', 1, 0) +
    1.19 * iif(Detections has 'Bkav:', 1, 0) +
    1.21 * iif(Detections has 'CAT-QuickHeal:', 1, 0) +
    0.49 * iif(Detections has 'CMC:', 1, 0) +
    1.55 * iif(Detections has 'ClamAV:', 1, 0) +
    2.03 * iif(Detections has 'CrowdSt